# Introduction

Here I'm testing the A2C on the taxi grid graph


In [3]:
import networkx as nx
import csv
import copy
import funcs as f
import numpy as np
import matplotlib.pyplot as plt
import gym
from agent_taxi import Agent
from agent_taxi import PolicyCab
from taxi_environment import Env
from keras.utils import to_categorical
%matplotlib inline

G = f.load_manhattan_graph()
G = nx.convert_node_labels_to_integers(G)  #nodes start at 1, want them to start at 0
state_zero = np.array([1 if i == 0 else 0 for i in range(G.number_of_nodes())])


def run(lr,gamma,penalty):
    
    
    #Environment
    state_zero = np.array([1 if i == 0 else 0 for i in range(G.number_of_nodes())])
    env = Env(G,state_zero)
    num_states = env.num_states
    num_actions = env.num_actions
    env.illegal_move_penalty = penalty

    #Agent
    agent = Agent(num_states, num_actions, lr, gamma)
    agent.memory_size = 1000

    scores = []
    EPISODES = 2000

    for e in range(1,EPISODES+1):
        state = state_zero
        state = np.reshape(state, [1, num_states])
        reward_sum = 0
        while agent.active_time < 100:

            # env.render()
            action = agent.act(state)
            next_state, reward = env.step(action,agent)
            reward_sum += reward
            next_state = np.reshape(next_state, [1, num_states])

            #We only want to remember action taken when looking
            if agent.state == 'serving':
                pass
            else:
                agent.remember(state[0], action, 1.0*reward)
            state = next_state
            

        #Learn & print results
        agent.train_models()
        tau = agent.find_tau()
        scores.append(tau)
        agent.reset_clocks()
        
    return scores

### Hyper parameter optimization

In [4]:
gammas = [0.01, 0.1, 0.9]
lrs = [0.0001,0.001,0.01,0.1]
penalty = -100

for lr in lrs:
    for gamma in gammas:
        scores = run(lr,gamma,penalty) # illegal move penalty
        print '(lr,gamma,mean,range) = ' + str((lr,gamma,np.mean(scores), max(scores) - min(scores)))

agent_taxi.py:73: RuntimeWarning: invalid value encountered in divide
  R /= np.std(R)
agent_taxi.py:92: RuntimeWarning: invalid value encountered in less
  action = np.random.choice(self.actions, p=probs)


(lr,gamma,mean,range) = (0.0001, 0.01, 0.202975, 1.0)
(lr,gamma,mean,range) = (0.0001, 0.1, 0.171235, 1.0)
(lr,gamma,mean,range) = (0.0001, 0.9, 0.182355, 1.0)
(lr,gamma,mean,range) = (0.001, 0.01, 0.20518, 1.0)
(lr,gamma,mean,range) = (0.001, 0.1, 0.18544, 1.0)
(lr,gamma,mean,range) = (0.001, 0.9, 0.18791, 1.0)
(lr,gamma,mean,range) = (0.01, 0.01, 0.174705, 1.0)
(lr,gamma,mean,range) = (0.01, 0.1, 0.18215, 1.0)
(lr,gamma,mean,range) = (0.01, 0.9, 0.211195, 0.99)
(lr,gamma,mean,range) = (0.1, 0.01, 0.17331, 1.0)
(lr,gamma,mean,range) = (0.1, 0.1, 0.19283, 1.0)
(lr,gamma,mean,range) = (0.1, 0.9, 0.1988, 1.0)


In [6]:
penalties = [-1000,-100,-10,0]
lr,gamma = 0.01, 0.01

for penalty in penalties:
    scores = run(lr,gamma,penalty) # illegal move penalty
    scores = scores[-1000:]  #discard the training
    print '(penalty,gamma,mean,range) = ' + str((penalty,gamma,np.mean(scores), max(scores) - min(scores)))

(penalty,gamma,mean,range) = (-1000, 0.01, 0.18819, 0.64)
(penalty,gamma,mean,range) = (-100, 0.01, 0.15662, 0.6)
(penalty,gamma,mean,range) = (-10, 0.01, 0.19398, 0.68)
(penalty,gamma,mean,range) = (0, 0.01, 0.19132999999999997, 0.71)
